# Variational Language Model
**QHack hackathon | 22-26 february 2021**\
*By `TeamX` Slimane Thabet & Jonas Landman* 


## Introduction
In this project, we developed a variational quantum algorithm for **Natural Language Processing** (NLP). Our goal is to **train a quantum circuit such that it can process and recognize words**. Applications varies from **word matching**, **sentence completion**, **sentence generation** and more.

---
#### Word encoding
Words are preprocessed using state-of-the art deep learning **word embedding** methods such as FastText. Then these embeddings arer cast down to few features using dimensionality reduction. For instance each word will be described as a vector of 8 dimensions. Using **Quantum Amplitude Encoding**, we can encode each word into a 3-qubits register. If a **sentence** is composed of $N$ words, and to represent it we propose to stack $N$ 3-qubits register sequentially.

#### Variational Circuit
We propose a new ansatz and training methodology to perform this NLP quantum learning:  
- The ansatz is composed of several layers of controlled rotations that mix the words between each other, and between themselves. 
- During the training, we will **mask one word randomly in each sentence**, by imposing its quantum register to  $|0\rangle$
- Using a **SWAP Test**, a supplementary word is then compared to the output register of the missing word (after the output of the ansatz). Therefore the cost function is the probability of output '0' on the swap test's ancillary qubit. We chose the supplementary word to be the missing word itself in order to drive the learning. 
- The goal of the training is to adjust the ansatz's parameters such that **the missing word is guessed**. 

<img src="circuit.png" alt="Drawing" style="width: 400px;"/>


#### Applications
With such a circuit trained, we can provide a new sentence with a missing word and compare it with all possible words in the "dictionary". We can generate artifical sentence by starting with only one word, or completing a sentence after its last words. 

<img src="sentence_generation.png" alt="Drawing" style="width: 300px;"/>



#### Performances
We consider $M$ sentences of $N$ words, each one encoded as $Q$ qubits. 
- **Number of qubits required**: One quantum circuit corresponds to one sentence plus an extra word and an ancillary qubit, therefore $Q*(N+1)+1$ qubits. E.g for a 4 words sentence with 3 qubits per words, we require 16 qubits. For a 5 words sentence with 4 qubits per words, we require 25 qubits. 
- **Number of trainable parameters**: The number of trainable parameters in the ansatz is around $Q*(1+N/2)*L$, where $L$ is the number of layers, on average (it depends of the parity of the number of words, and number of qubits). E.g for a 4 words sentence with 3 qubits per words and 3 layers, we require 27 parameters.

We can use AWS SV1 for parallelizing the gradient during the training. But the computational cost remains high due to the number of sentences and the total number of words in the dictionary. 

#### Datasets
We propose 3 differents datasets to train and test our algorithm
- **IMDB Dataset** composed of (?) sentences and (?) words in total
- **Newsgroup Dataset** composed of (?) sentences and (?) words in total
- An **synthetic dataset** of 'dummy' sentences with small number of sentences and words, for performance limitation and grammatical simplicity


#### Code architecture
- The **Pennylane** variational ansatz are defined in `utils.py`
- The NLP preprocessing using FastText is made in `embeddings.ipynb` and generate readable file as `embeddings.npy`, `sentences.npy` etc.
- In `config.py` are defined the global configurations such as the number of words, of qubit per words, and the number of layers per ansatz.
- In this notebook, we train the quantum variational circuit and test applications

---

##### Imports

In [1]:
import pennylane as qml
from pennylane import numpy as np
from config import config
from utils import circuit_final, encode_words
import torch
from torch.autograd import Variable
from sklearn.decomposition import PCA
import pickle
from time import time

qml.enable_tape()
num_words = config['NUM_WORDS']
qbits_per_word = config['QUBITS_PER_WORDS']
num_layers = config['NUM_LAYERS']


my_bucket = f"amazon-braket-edb2457fc968" # the name of the bucket
my_prefix = "Variational-NLP" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"

## Sentence preprocessing
Load embeddings and sentence, apply dimensionality reduction, randomly defined missing words in each sentence, generate the input structure for our variational quantum circuit. 

In [2]:
n_dim = 2**qbits_per_word
max_length = num_words

embeddings = np.load("newsgroup/embeddings.npy")
sentences = np.load("newsgroup/sentences.npy").astype(int)
labels = np.load('newsgroup/labels.npy')

np.random.seed(143)
missing_word = np.random.randint(0, num_words, size=len(sentences)).astype(int)#.numpy()

norms = np.linalg.norm(embeddings, axis=1)
pca = PCA(n_dim)
embeddings_reduced = np.zeros((embeddings.shape[0], n_dim))
embeddings_reduced[norms>0] = pca.fit_transform(embeddings[norms>0])

norms_reduced = np.linalg.norm(embeddings_reduced, axis=1).reshape(-1,1)
embeddings_reduced_norm = np.zeros_like(embeddings_reduced)#.numpy()
embeddings_reduced_norm[norms>0] = embeddings_reduced[norms>0] / np.repeat(norms_reduced[norms>0], n_dim, axis=1)

embeddings_reduced_norm.requires_grad = False
sentences_truncated = sentences[:,0:max_length]
sentences_truncated.requires_grad = False

missing_word.requires_grad = False

all_indices = np.repeat(np.arange(max_length).reshape((1,-1)), len(sentences), axis=0).astype(int)#.numpy()
for i in range(len(sentences)):
    all_indices[i, missing_word[i]] = max_length
all_indices.requires_grad = False

with open('newsgroup/vocab.p', 'rb') as readfile:
    vocab = pickle.load(readfile)

word_to_id = vocab
id_to_word = {value:key for key,value in vocab.items() if np.linalg.norm(embeddings_reduced_norm[int(value)])>0}

word_indices = list(id_to_word.keys())

Here is what the data look like

In [3]:
print("Original sentence: ", ' '.join(id_to_word[int(id)] for id in sentences[10]))
print("Truncated sentence: ", ' '.join(id_to_word[int(id)] for id in sentences_truncated[10]))

Original sentence:  known quite earth actually spherical anyone make accurate actual shape configuration long
Truncated sentence:  known quite earth actually


Let us look at the word embeddings

In [4]:
word1 = 'medicine'
word2 = 'disease'
word3 = 'january'

id1, id2, id3 = vocab[word1], vocab[word2], vocab[word3]
e1, e2, e3 = embeddings[[id1, id2, id3]]
er1, er2, er3 = embeddings_reduced_norm[[id1, id2, id3]]

print("Similarity between '", word1, "' and '", word2, "' in original fasttext embedding: ", np.abs(np.dot(e1, e2))/(np.linalg.norm(e1)*np.linalg.norm(e2)))
print("Similarity between '", word1, "' and '", word3, "' in original fasttext embedding: ", np.abs(np.dot(e1, e3))/(np.linalg.norm(e1)*np.linalg.norm(e3)))
print("")
print("Similarity between '", word1, "' and '", word2, "' in reduced normalized embedding: ", np.abs(np.dot(er1, er2)))
print("Similarity between '", word1, "' and '", word3, "' in reduced normalized embedding: ", np.abs(np.dot(er1, er3)))


Similarity between ' medicine ' and ' disease ' in original fasttext embedding:  0.5201646458004966
Similarity between ' medicine ' and ' january ' in original fasttext embedding:  0.12138553809780184

Similarity between ' medicine ' and ' disease ' in reduced normalized embedding:  0.9751034457579837
Similarity between ' medicine ' and ' january ' in reduced normalized embedding:  0.2794989681225861


## Model training
The circuit is defined as `circuit_final(params, wires, num_layers, target_word)` in `utils.py`

##### Set Up

In [5]:
n_wires = qbits_per_word * (max_length+1) + 1

#dev_remote = qml.device(
 #   "braket.aws.qubit",
  #  device_arn=device_arn,
   # wires=n_wires,
   # s3_destination_folder=s3_folder,
   # parallel=True
#)

dev_local = qml.device("default.qubit", wires=n_wires)

dev = dev_local
#dev = dev_remote

@qml.qnode(dev)
def compute_overlap_words(parameters, embeddings, indices, target_word, wires=dev.wires):
    encode_words(embeddings, indices)
    params = [(parameters[:,0,i], parameters[:,1::,i]) for i in range(num_layers)]
    circuit_final(params, wires, num_layers, target_word)
    return qml.expval(qml.PauliZ(wires[-1]))


def cost(parameters, sentences, missing_words):
    cost = 0    
    for i,sentence in enumerate(sentences):
        embeddings = embeddings_reduced_norm[sentence]
        indices = all_indices[i]
        m_w = missing_words[i]
        cost += compute_overlap_words(parameters, embeddings, indices, target_word = m_w)
    return cost

##### Training

In [6]:
batch_size = 10
epochs = 2
N_batches = len(sentences_truncated)//batch_size

parameters = np.random.rand(qbits_per_word, int(np.ceil(num_words/2))+1, num_layers)

opt = qml.AdamOptimizer(stepsize=0.01)

losses = []

for epoch in range(epochs):
    for i in range(N_batches):
        t0 = time()
        batch = np.arange(i*batch_size, (i+1)*batch_size).astype(int)
        
        def cost_batch(parameters):
            return cost(parameters, sentences_truncated[batch], missing_word[batch])
        
        parameters = opt.step(cost_batch, parameters)
        t1 = time()
        print('Time: ', t1-t0)

DeviceError: Operation QubitStateVector cannot be used after other Operations have already been applied on a default.qubit.autograd device.

## Applications

In [ ]:
def get_most_probable_word(sentence, position, look_in=None):
    assert position<num_words
    if look_in is None:
        look_in = np.arange(len(word_indices)).astype(int)
    indices = []
    for i in range(num_words):
        if i!=position:
            indices.append(i)
    indices.append(num_words)       
    probas = []
    embeddings_input = embeddings_reduced_norm[sentence]
    for i,index in enumerate(word_indices[look_in]):
        embeddings = np.concatenate([embeddings_input, embeddings_reduced_norm[index].reshape((1,-1))], axis=0)
        probas.append(float(compute_overlap_words(parameters, embeddings, indices, target_word = position)))
    return probas

input_sentence = 'january [mask] within million march 1986 spacecraft'

list_words = input_sentence.split(' ')
list_index = []
missing_index = 0
for i,word in enumerate(list_words):
    if word=='[mask]':
        missing_index = i
    else:
        list_index.append(vocab[word])

np.random.seed(23)
look_in = np.random.randint(len(word_indices), size=10).astype(int)
p = get_most_probable_word(list_index, 4, look_in=look_in)

print("The 5 most probable words are: ")
print(' '.join(id_to_word[int(look_in[i])] for i in np.argsort(p)[::-1][0:5]))